# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science 

# Lab 10: Bagging, Random Forests, & Interpreting ML Models


**Harvard University**<br/>
**Fall 2024**<br/>
**Instructors**: Pavlos Protopapas and Natesh Pillai<br/>
<hr style='height:2px'>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

In [ ]:
# Supress pesky warnings (probably bad practice)
from warnings import simplefilter
simplefilter(action="ignore")

# Table of Contents
- Cambridge House Prices Regression Dataset: EDA
- Single Tuned Tree
- Bagging Ensemble
- Out-of-Bag Error (OOB)
- Random Forest
- Model Interpretation
    - Interpretation Through Predictions
    - Feature Importance 
    - Permutation Importance
    - LIME 

## Regression Dataset: Cambridge housing prices

In [ ]:
homes = pd.read_csv("data/cambridgehomes.csv")
homes_raw = homes

In [ ]:
# Bare minimum preprocessing
homes["zip"] = homes["zip"].astype(str)
homes['lotsize'] = homes['lotsize'].fillna(0)
homes['hoa'] = homes['hoa'].fillna(0)
type_dummies = pd.get_dummies(homes['type'], drop_first=True)
homes = pd.concat([homes,type_dummies],axis=1)
homes.dtypes

In [ ]:
y = np.log2(homes['price']/1000)
X = homes[['multifamily', 'singlefamily', 'townhouse',
                'sqft',  'dist', 'beds', 'baths', 'year','hoa','lotsize']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=109)

In [ ]:
X_train.head()

In [ ]:
y_train.plot(kind='hist', title=r"$\log_2$(price)");

In [ ]:
nrow = 2; ncol = 5;
fig, axs = plt.subplots(nrows=nrow, ncols=ncol, figsize=(20, 8))

fig.suptitle('EDA for Home dataset', fontsize=16)

for ax, column in zip(axs.reshape(-1), X_train.columns):
    ax.scatter(X_train[column], y_train, alpha=0.2)
    ax.set_xlabel(f'{column}', fontsize=14)
    ax.set_ylabel('Home Price', fontsize=12)

## Tuned Decision Tree

Our first model is a single decision tree. We use CV to select the best value for the `max_leaf_nodes` stopping criteria.

In [ ]:
dtree = DecisionTreeRegressor()
params = {'max_leaf_nodes': [10, 15, 20, 25, 30]}
# grid = GridSearchCV(dtree, params, cv=5, scoring='neg_mean_squared_error')
grid = GridSearchCV(dtree, params, cv=5, scoring='r2')
grid.fit(X_train, y_train)

In [ ]:
best_tree = grid.best_estimator_
best_tree

In [ ]:
plt.figure(figsize=(20,20))
tree.plot_tree(best_tree, feature_names=X_train.columns);

In [ ]:
dt_train_r2 = best_tree.score(X_train, y_train)
dt_cv_r2 = grid.cv_results_['mean_test_score'].max()
dt_test_r2 = best_tree.score(X_test, y_test)
print(f"Train R^2:\t{dt_train_r2:.4f}")
print(f"CV R^2:\t\t{dt_cv_r2:.4f}")
print(f"Test R^2:\t{dt_test_r2:.4f}")

## Bagging

<img src="fig/bagging1.png" alt="bagging" width="60%"/>

In this section  we'll

    1. Fit a Bagging classifer using multiple bootstrapped datasets and do majority voting. 
    2. Evaluate the model on OOB and feature importance.
    
Hopefully after this lab you will be able to answer the following questions: 

- What is the main idea behind bagging?
- Why does bagging help with overfitting?
- Why does bagging help to built more expressive trees?
- What is OOB? How should we use it?
- How can we measure feature importance with trees?


*QUESTION:* Where does the word *"Bagging"* come from?

**Some Theory: What is bagging?**
  1. Bootstrapping: resample with replacements to get different datasets and built different models.
  2. Do something smart to combine the different models.
  
One way to adjust for the high variance of the output of an experiment is to perform the experiment multiple times and then average the results. 

 1. **Bootstrap:** we generate multiple samples of training data, via bootstrapping. We train a full decision tree on each sample of data. 
 2. **AGGregatiING** for a given input, we output the averaged outputs of all the models for that input. 
 
This method is called **Bagging: B** ootstrap + **AGG**regat**ING**. 

-----------

Let's bootstrap our training dataset to create multiple datasets and fit Decision Tree models to each.

(Resampling: we showed live that different samples give different results for things like sums, varying more when the things we sum over have high variance themselves.)



### Bagging from Scratch

In [ ]:
n_trees = 1000 # This can be tuned
# Create a decision tree to serve as the base model in the bagged ensemble
best_params = best_tree.get_params()
# model = DecisionTreeRegressor().set_params(**best_params)
# model = DecisionTreeRegressor(max_leaf_nodes=int(2*best_params['max_leaf_nodes']))
model = DecisionTreeRegressor(max_depth=None)

# Initializing arrays to store results
predictions_train = np.zeros( shape = (X_train.shape[0], n_trees))
predictions_test  = np.zeros( shape = (X_test.shape[0],  n_trees))

# Conduct bootstraping iterations
for i in range(n_trees):
    boot_idx = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=True)
    boot_X = X_train.iloc[boot_idx]
    boot_y = y_train.iloc[boot_idx]

    # Fir on bootstrap
    model.fit(boot_X, boot_y)  

    # Store train and test predictions
    predictions_train[:,i] = model.predict(X_train)   
    predictions_test[:,i] = model.predict(X_test)
    
# Make Predictions Dataframe
columns = ["Bootstrap-Model_"+str(i+1) for i in range(n_trees)]
predictions_train = pd.DataFrame(predictions_train, columns=columns)
predictions_test = pd.DataFrame(predictions_test, columns=columns)

In [ ]:
# Aggregated predictions evaluation
bag_train_r2 = r2_score(y_train, predictions_train.mean(axis=1))
bag_test_r2 = r2_score(y_test, predictions_test.mean(axis=1))
print(f"Train R^2:\t{bag_train_r2:.4f}")
print(f"Test R^2:\t{bag_test_r2:.4f}")


**Q:** 🤔 Are these bagging models independent of each other, can they be trained in a parallel fashion?

---

### Out-of-Bag Score (OOB)

*QUESTION:* 
- What is out-of-bag (OOB) error? 
- How can we take advantage if it to improve our model's performance?
- Why OOB is a great method?

<img src="fig/oob.png" alt="tree_adj" width="60%"/>

Out-of-bag (OOB) error/Out-of-bag estimate is validated method availble to us whenever we use a bagged ensemble.
OOB samples can be seen as the validation set, generated by bootstrap process. So, we don’t need to do the explicit setup another validation set. Compared to CV, it has the following advantages.

There is less computational cost for OOB error compared to traditional CV. 


In [ ]:
n_trees = 1000 # This can be tuned
# Create a decision tree to serve as the base model in the bagged ensemble
best_params = best_tree.get_params()
# model = DecisionTreeRegressor().set_params(**best_params)
# model = DecisionTreeRegressor(max_leaf_nodes=int(2*best_params['max_leaf_nodes']))
model = DecisionTreeRegressor(max_depth=None)

# Initializing arrays to store results
predictions_train = np.zeros( shape = (X_train.shape[0], n_trees))
predictions_test  = np.zeros( shape = (X_test.shape[0],  n_trees))
predictions_oob = np.zeros(shape = (X_train.shape[0], n_trees))*pd.NA # fill /w NA by default

# Conduct bootstraping iterations
for i in range(n_trees):
    boot_idx = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=True)
    boot_X = X_train.iloc[boot_idx]
    boot_y = y_train.iloc[boot_idx]
    
    model.fit(boot_X, boot_y)  
    predictions_train[:,i] = model.predict(X_train)   
    predictions_test[:,i] = model.predict(X_test)
    
    # Get OOB samples
    oob_mask = ~np.isin(np.arange(X_train.shape[0]), boot_idx)
    X_oob = X_train[oob_mask]
    y_oob = y_train[oob_mask]
    oob_p = model.predict(X_oob)
    predictions_oob[oob_mask, i] = oob_p # Update prediction results of this tree
    
# Make Predictions Dataframe
columns = ["Bootstrap-Model_"+str(i+1) for i in range(n_trees)]
predictions_train = pd.DataFrame(predictions_train, columns=columns)
predictions_oob = pd.DataFrame(predictions_oob, columns=columns)
predictions_test = pd.DataFrame(predictions_test, columns=columns)

In [ ]:
bag_train_r2 = r2_score(y_train, predictions_train.mean(axis=1))
bag_test_r2 = r2_score(y_test, predictions_test.mean(axis=1))
bag_oob_r2 = r2_score(y_train, predictions_oob.mean(axis=1))
print(f"Train R^2:\t{bag_train_r2:.4f}")
print(f"OOB R^2:\t{bag_oob_r2:.4f}")
print(f"Test R^2:\t{bag_test_r2:.4f}")

---

### SKLearn's BaggingRegressor

From SKlearn's [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html):

"A Bagging regressor is an ensemble meta-estimator that fits base regressors each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. Such a meta-estimator can typically be used as a way to reduce the variance of a black-box estimator (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it."

The most important arguments are `estimator` which defines the base estimators from which the bagged ensemble is composed and well as `n_estimators`, the number of estimators in the ensemble.
If you want access to the `oob_score_` attribute on the fitted BaggingRegressor you will also need to set `oob_score=True` since it is `False` by default to save a bit of computation.

The individual estimators in the ensemble can be accessed by the `estimators_` attribute.

`warm_start=True` will cause the BaggingRegressor to re-use the previous estimators when re-fitting with a larger setting of `n_estimators`. This saves computation as you only need to fit the new trees being added to the ensemble.

In [ ]:
# Define a BaggingRegressor using a DecisionTreeRegressor as its base estimator
bag = BaggingRegressor(estimator=DecisionTreeRegressor(),
                  n_estimators=100,
                  oob_score=True)

In [ ]:
# Fit on train
bag.fit(X_train, y_train)

In [ ]:
# Inspect OOB accuracy
print("OOB ACC:", bag.oob_score_)

In [ ]:
# Inspect OOB accuracy
print("OOB ACC:", bag.oob_score_)

In [ ]:
# Inspect test accuracy
print("Test ACC:", bag.score(X_test, y_test))

<div class="alert alert-success">
    <strong>🏋🏻‍♂️ ACTIVITY:</strong> Number of Estimator's Affect on BaggingRegressor </div>  

- Create a visualization showing how the number of estimators effects the bagging regressor's train and OOB scores.
- You should use the default, 'full-depth' trees as your base estimators.
- There are many ways to accomplish this!

**Hint:** You may want to adjust the y-axis limits of your plot. Why is this necessary?

In [ ]:
# your code here


**Q**: 🤔 Can you think of a way to further reduce the variance of the ensemble model?

### Weaknesses of Bagging

Bagging is a *greedy* algorithm. What does this mean?\
We always choose to split on the feature with the most impact: i.e., the most information gain.

$\bullet$ Because of their greedy nature, bagging ensembles are very likely to be correlated, especially in the shallower nodes of the individual decision trees.

### Why are decision trees greedy?

$\bullet$ Finding the optimal decision trees is an NP-complete problem, so in practice this is infeasible.

$\bullet$ Thus decision trees are **hueristic algorithms**. Hueristic algorithms are designed to solve problems by sacrificing optimality, accuracy, or precision in favor of speed. Hueristic algorithms are often used to solve NP-complete problems.

---

## Random Forest

Random Forest is a modified form of bagging that creates ensembles of independent decision trees. 
To *de-correlate the trees*, we: 
1. train each tree on a new bootstrapped sample of the full training set (same as in bagging) 
2. for each tree, at each split, we **randomly select a set of 𝐽′ predictors from the full set of predictors.** (not done in bagging)
3. From among the 𝐽′  predictors, we select the optimal predictor and the optimal corresponding threshold for the split. 

<font color='red'> *Question:* </font>  Why would this second step help (only considering random sub-group of features)?



The averaging in Random Forest is able to reduce even more variance because the resulting trees are less correlated.\
This reduces overfitting as seen in the smaller gap between the train and OOB scores.

Fit a RandomForestRegressor using the same `max_depth` and `n_estimators` as your final, large bagging model. But here we will set the `max_features` parameter to 'sqrt' to only consider a random set of predictors of size $\sqrt{J}$ at each split, where $J$ is the original number of predictors.

<font color='red'> NOTE: </font>  Please referring to the SKlearn documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
rf = RandomForestRegressor(max_depth=None,
                           n_estimators=1000,
                           max_features='sqrt',
                           oob_score=True,
                           random_state=109)
rf.fit(X_train, y_train)
print(f'The train score is: {rf.score(X_train, y_train):.4f}')
print(f'The test score is: {rf.score(X_test, y_test):.4f}')
print(f'The OOB score is: {rf.oob_score_:.4f}')

### Tuning a Random Forest

**Which parameters need to be tuned for a random forest model?**. Hint: there are 2 (or maybe 3)


Use Cross-Validation (via `GridSearchCV`) or out-of-bag error (similar to the bagged model above) to tune a random forest model.  Consider 'min_samples_split' in the set of [2,5,10,20] and 'max_features' in the set of [2,5,8].

<font color='red'> NOTE: </font> The choices of parameters under param_grid is based on the attributes in your estimator. For example, if your estimator is RandomForestRegressor, the choices of parameters are the attributes of the RandomForestRegressor, and you should refer to the SKlearn RandomForestRegressor documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
rf2 = GridSearchCV(estimator=RandomForestRegressor(random_state=109),
                  param_grid={'min_samples_split': [2,5,10,20],
                              'max_features': [2,5,8]},
                  cv=5)
rf2.fit(X_train, y_train)

In [ ]:
# Just refitting outside of GridSearchCV so that we can extract Variable Importances later
rf_tuned = RandomForestRegressor(max_features=rf2.best_params_['max_features'],
                                 min_samples_split=rf2.best_params_['min_samples_split'],
                                 random_state=109).fit(X_train, y_train)

In [ ]:
rf2.best_params_

**Q**: 🤔 A bagging model is just a special case of a random forest model.  Discuss.

**Q**: 🤔 **Why** do random forests improve predictions over a bagging model?  Under what conditions (think relationships among all of your variables) will this improvement be large?  When will it be small or perform even worse?

**Q**: 🤔 A bagging model is just a special case of a random forest model.  Discuss.

**Q**: 🤔 **Why** do random forests improve predictions over a bagging model?  Under what conditions (think relationships among all of your variables) will this improvement be large?  When will it be small or perform even worse?

---

## Model Interpretability 🔎

- As a data scientist, you need to know ins and outs of machine learning models.
- Often you need to provide a **trustworthy, transparent, and accountable explanation** to stakeholders beyond the final evaluation metrics (e.g., $R^2$, MSE, Accuracy, ROC-AUC score, etc.).
- To provide a good justification, we would do well to have an easily generalizable framework to explain a machine learning model.

But there are inherent trade-off between model interpretability and accuracy.\
And there are many ways to explain the machine learning models:
- Use only interpretable models
    - Linear regression, logistic regression, decision trees, etc.
    - But these are often times too simple and don't perform well!
- Use model specific interpretation methods
    - Lacks generalization because it is model dependent!
- Use **model-agnostic methods**

#### Fitting 5 tree-based models to predict `log2(price)` from `sqft` alone

The simplest (and best?) way to interpret a machine learning model is through the plotting the predictions! For a regression-type problem, that means visualizing $\hat{y}$ vs. $x_j$.  For a calssification problem, that means visualizing $\hat{P}(y=1)$ vs. $x_j$ or visualizing the decision boundary vs. $x_j$ and $x_{j`}$.

When there is just a single predictor in the model, then plotting the predictions is easy (see below)!  

In [ ]:
# Range of observed sqft predictor
dummy_x = np.arange(np.min(homes['sqft']), np.max(homes['sqft']),1)

plt.plot(X_train['sqft'], y_train,'.',alpha=0.5)

bag_simple = RandomForestRegressor(min_samples_split=50,
                           n_estimators=500,
                           max_features=1,
                           oob_score=True,
                           random_state=109)
bag_simple.fit(X_train[['sqft']], y_train)

plt.plot(dummy_x, bag_simple.predict(dummy_x.reshape(-1,1)), 
             label="bag_simple", alpha=1, lw=2)

for i in [5, 3, 2, 1]:
    dtree = tree.DecisionTreeRegressor(max_depth=i)
    dtree.fit(X_train[['sqft']],y_train)
    plt.plot(dummy_x, dtree.predict(dummy_x.reshape(-1,1)), 
             label=("max depth = "+str(i)), alpha=0.7, lw=2)

plt.legend();

**Q**: 🤔 Which of these models are overfit?  Which are potential underfit?  What do the models say about the relationship between `log2(price)` and `sqft`?  Why isn't there technically a random forest model?

#### Fitting Four multiple-predictor ML Models

When there are multiple predictors, things get tough! We fit 4 different tree-based models using the 10 predictor $X$ from before:

Start with 2 decision tree to add to our previous ensemble models: the tuned random forest model (`rf2`) and the untuned bagged model (`bag`)

In [ ]:
# fit a possibly underfit (depth = 3) decision tree regressor
dt3 = tree.DecisionTreeRegressor(max_depth = 3)
dt3.fit(X_train,y_train);

# fit a well-tuned decision tree regressor using our best params from before
dt_tuned = tree.DecisionTreeRegressor(max_leaf_nodes=best_params['max_leaf_nodes'])
dt_tuned.fit(X_train,y_train);

# and the other two models are your well-tuned Random Forest (`rf2`) and your untuned `bag` model

We now explore the relationships by plotting the predictions (we'll stick with sqft for now).  

**Q**: 🤔 How can we generate these prediction plots?  What's the tricky detail we need to worry about?

**Q**: 🤔 How do tree-based models handle interactions?  Why does this make sense?

In [ ]:
# Create the data frame of means to do the prediction
means1 = X_train.mean(axis = 0)
means_df = (means1.to_frame()).transpose()

# Do the prediction across range of observed sqft
dummy_x = np.arange(np.min(homes['sqft']),np.max(homes['sqft']),1)
means_df  = pd.concat([means_df]*dummy_x.size,ignore_index=True)
means_df['sqft'] = dummy_x
yhat_rf = rf2.predict(means_df)

In [ ]:
#plots yhat vs. sqft at means of the other predictors
plt.scatter(X_train['sqft'],y_train)
plt.plot(dummy_x, bag_simple.predict(dummy_x.reshape(-1,1)), 
             label="bag_simple",color='#000000')
plt.plot(means_df['sqft'],yhat_rf,
         label="rf_tuned",color='#A51C30')
plt.title("Predicted log2(price) vs. sqft from RF in train")
plt.legend();

*Note: Harvard Colors: https://seas.harvard.edu/office-communications/brand-style-guide/color-palette

**Q**: 🤔 Describe what you see.  How does the RF model `log2(price)`s relationship with `sqft`? How is this different from the *simple* RF/bagging model?  Why is this not surprising?

**$^*$Hint**: Think simple vs. multiple regression.

In [ ]:
# Create the data frame of means for condos and separately 
# for multi-family homes to do the prediction
means_condo = X_train[(X_train['multifamily']+
                       X_train['singlefamily']+
                       X_train['townhouse'])==0].mean(axis = 0)
condo_df = (means_condo.to_frame()).transpose()
condo_df  = pd.concat([condo_df]*dummy_x.size,ignore_index=True)
condo_df['sqft'] = dummy_x

means_multi = X_train[X_train['multifamily']==1].mean(axis = 0)
multi_df = (means_multi.to_frame()).transpose()
multi_df  = pd.concat([multi_df]*dummy_x.size,ignore_index=True)
multi_df['sqft'] = dummy_x


# Do the prediction at all observed sqft for each home type
yhat_rf_condo = rf2.predict(condo_df)
yhat_rf_multi = rf2.predict(multi_df)

In [ ]:
#plots yhat vs. sqft at means of the other predictors for condos vs. multifamily homes
plt.scatter(X_train['sqft'],y_train)
plt.plot(condo_df['sqft'],yhat_rf_condo,
         label='condo',color='#A51C30')
plt.plot(multi_df['sqft'],yhat_rf_multi,
         label='multifamily',color='#000000')
plt.title("Predicted log2(price) vs. sqft from RF in train")
plt.legend();

**Q**: 🤔 Describe what you see.  How does the RF model `log2(price)`s relatoinship with `sqft`?  What does this say about any interaction effects involving `sqft`?

### Feature Importance (MDI)

Fill in the blanks below to calculate the variable importances from the 4 untuned models above.

In [ ]:
X.columns[np.flip(np.argsort(dt3.feature_importances_))]

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(24,6))
models = {'depth-3 tree': dt3,
          'tuned tree': dt_tuned,
          'bagged': bag,
          'random forest': rf_tuned}

num_features = 10 
for i, (name, model) in enumerate(models.items()):
    # calculation is more manual for the bagged model
    if name == 'bagged':
        imporances = np.array([t.feature_importances_ for t in model.estimators_]).mean(axis=0)
    else:
        importances = model.feature_importances_
    order = np.argsort(importances)[-num_features:]
    axs[i].barh(range(num_features), importances[order], tick_label=X_train.columns[order]);
    axs[i].set_title(f"Relative Variable Importance for {name}")

plt.tight_layout()

**Q**: 🤔 How do these variable importance measures compare for these 4 models?  Which predictor is most important in general?  How is it related to `price`? 

**What other approaches can be taken to measure variable importance?**

One alternative for random forest:
- Record the prediction accuracy on the *oob* samples for each tree.
- Randomly permute the data for column $j$ in the *oob* samples, then record the accuracy again.
- The decrease in accuracy as a result of this permuting is averaged over all trees, and is used as a measure of the importance of variable $j$ in the random forest. 

This idea of re-permuting a variable and *refitting* a model to see how much more
poorly it performs is called **permutation feature importance**.

---

### Permutation Importance

Keep in mind that when two features are correlated and one of the features is permuted, the model will still have access to the feature through its correlated feature.

**Q:** 🤔 What is the one glaring disadvantage to the permutation approach? ⏲️

**Q:** 🤔 This method is often preferred to the standard, MDI feature importance approach shown above, why?

SKLearn provides a built-in implementation of permutation importance that we can use to evaluate our model:

In [ ]:
from sklearn.inspection import permutation_importance

# Calculate permutation importance
pi_results = permutation_importance(rf, X_test, y_test, n_repeats=50, random_state=42)

# Create a DataFrame to display results
pi_data = {
    'importance_mean': pi_results['importances_mean'],
    'importance_std': pi_results['importances_std'],
    'feature': X_test.columns
}
pi_df = pd.DataFrame(pi_data).sort_values('importance_mean', ascending=False)

# Sort by importance for plotting
pi_df_sorted = pi_df.sort_values('importance_mean', ascending=True).reset_index(drop=True)
pi_df_sorted

The results show:
- importance_mean: Average decrease in model score when the feature is permuted
- importance_std: Standard deviation of the importance across permutations
- feature: Name of the feature being evaluated

Higher importance values indicate features that, when randomly shuffled, lead to larger decreases in model performance.

In [ ]:
# Feature importance barplot with error bars
plt.figure(figsize=(10, 6))
plt.barh(range(len(pi_df_sorted)), pi_df_sorted['importance_mean'],
         xerr=pi_df_sorted['importance_std'],
         capsize=5, alpha=0.8, color='#A51C30')  # Harvard Crimson color!
plt.yticks(range(len(pi_df_sorted)), pi_df_sorted['feature'])
plt.xlabel('Permutation Importance (decrease in $R^2$ score)')
plt.title('Permutation Feature Importance with Standard Deviation')

# Add grid for easier comparison
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

**Q**: 🤔 How do the permutation importance measures compare to the default variable importance in the random forest?  

---

Variable Importance is great! It tells you what features are important in shaping the model predictions.\
But what is missing?
- It does not give any measure for *how* the predictors are related to the response (positive, negative, quasi-linear, curved, interactions, etc.).
- This is where the parametric model wins out! Inference and interpretations are much easier and is the whole point of such models.

**Q:** 🤔 What can we do to measure these relationships in a machine learning or nonparametric model? What did we do with k-NN? 

Well, we could just plot the predictions! Easy with 1 predictor, but what if we have hundreds?

How do I extract the relationship between a given predictor and the response when, in the model, it is embedded in the context of the other predictors?\
We could hold all the other variables constant!

What needs to be done algorithmically to put this in practice?
Set the other predictors equal to *something* and only vary the predictor of interest)

---

### LIME 

The above stratagies attempt to give us a **global** explanation of our model's predictions.
But we may be interested in a **local** explanation. That is, why did our blackbox model give *a particular* observation the prediction it did?

A method called **LIME** was proposed in the paper ["Why Should I Trust You?": Explaining the Predictions of Any Classifier](https://arxiv.org/pdf/1602.04938v3.pdf), and it attempts to do just this. So what is LIME?

**Locally**: The explanation should be able to explain how the model behaves for individual observations.\
**Interpretable:** The explanation must be easy to understand by humans (but may depend on the target audience).\
**Model-agnostic:** The method should be able to explain any model.\
**Explanations:** self-explanatory 😁. This is the whole goal!

The approach is to use a directly interpretable model (e.g., a linear model) to help explain a model that is not directly interpretable.

**LIME ALGORITHM**
1. Select your observation of interest for which you want to have an explanation of its black box prediction.
2. Randomly generate points all over the feature space (sample X values from a Normal distribution inferred from the training set)
3. Get the black box predictions for these new points.
4. Weight each of the generated points based on their proximity to the point of interest using a kenerl function (e.g., exponential, Gaussian, etc.)
5. Fit a *weighted*, interpretable model on the synthetic data and the original model's predictions.
6. Explain the prediction by interpreting the local model.

This method can be applied to tabular data, text, and images!

**Toy Example: Classification Problem**

<img src='fig/LIME_plot.png' width='400px'>

In the plot above, the colored regions represent either side of the original model's decision boundary.\
The large red marker is our point of interest.\
The other points are the generated data. Their color represents the *original* model's predictions and their size represents their weight based on their distance from the point of interest.\
The dashed line is the predictions of the interpretable surrogate model.

The **objective** can be expressed as:

<img src='fig/LIME_eq.png'>

The explanation model for observation $x$ is the model $g$ (e.g., logistic regression model) that minimizes loss $L$ (e.g., cross-entropy), which measures how close the explanation is to the prediction of the original model $f$ (e.g., random forest), while the model complexity $\Omega$ is kept low (e.g., prefer fewer features).\
$G$ is the family of possible explanations.\
$\pi_x$ is a proximity measure defining a neighborhood around instance $x$ we consider for an explanation.

There are several LIME implementations for Python. We'll be using one simply called [lime](https://github.com/marcotcr/lime).

Because we're working with tabular data we'll be using [LimeTabularExplainer](http://gael-varoquaux.info/interpreting_ml_tuto/content/02_why/04_black_box_interpretation.html).

In [ ]:
# !pip install lime

In [ ]:
import lime

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(X_train.values,
                                 feature_names=X_train.columns,
                                 class_names = [0,1],
                                 mode='regression')

In [ ]:
idx = 109

exp = explainer.explain_instance(X_train.values[idx], 
                                 rf.predict, 
                                 num_features = 10)

print('Observation #: %d' % idx)
print('Predicted Price (in thousands) =', 2**(rf.predict(X_train)[idx]))
print('Actual Price (in thousands): %s' % 2**(y_train.iloc[idx]))

In [ ]:
### Plot the results
exp.as_pyplot_figure();

In [ ]:
# change the observation number and see what changes.
idx = 209

exp = explainer.explain_instance(X_train.values[idx], 
                                 rf.predict, 
                                 num_features = 10)

print('Observation #: %d' % idx)
print('Predicted Price (in thousands) =', 2**(rf.predict(X_train)[idx]))
print('Actual Price (in thousands): %s' % 2**(y_train.iloc[idx]))

In [ ]:
### Plot the results
# exp.as_list()
exp.as_pyplot_figure();

We also have a `show_in_notebook` methode but the pyplot figure one above is more concise.

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)

You can also print the explanation as list.

In [ ]:
exp.as_list()

**Q:** 🤔 Interpret the LIME results above.  Do they agree with the other interpretations for the random forest model seen so far?

<div class="alert alert-success">
    <strong>🏋🏻‍♂️ TEAM ACTIVITY:</strong> Inspect the Worst Prediction</div>  

**Instruction:**
- Which observation in the test data does the random forest get *most wrong*?
    - Think about how you would determin this
- Use LIME to interpret this bad prediction.
    - Do we have any insight into what is driving the mistake or not?

In [ ]:
# your code here


#### Potential Issues

- The surrogate is fit to **randomly generated data**. And so interpretations can be *unstable*, changing with each run.
- The local approximation is highly sensitive to the choice of **kernel width**.

When LIME was initially proposed, the kernel used to define the neighborhood near the point of interest was selected using heuristics.\

<img src='fig/SHAP_kernel.png' width='700px'>

<p style="font-size:11px">Image by <a href="https://towardsdatascience.com/lime-explain-machine-learning-predictions-af8f18189bfe">Giorgio Visani</a></p>

Though there has been recent work on LIME's instability and sensitivity to kernel width. [OptiLIME](https://arxiv.org/pdf/2006.05714.pdf) proposes a more principled way.\
They have an open-source implementation on [Github](https://github.com/giorgiovisani/lime_stability/tree/master/OptiLIME).

## Conclusion

We looked at how we can use prediciton plots, variable importance, permutation importance, and LIME to explain machine learning models in model-agnostic way.

But there will always remain a tension between explainability/interpretability and accuracy of statistical and machine learning models.


## References

Interpretable ML: https://christophm.github.io/interpretable-ml-book

ELI5: https://eli5.readthedocs.io/en/latest/index.html

LIME: https://github.com/marcotcr/lime


🌈 **The End**